In [38]:
import time
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
import time
from IPython.display import clear_output
import requests
import pandas as pd
import pandas_ta as ta
try:
    import talib
    USE_TALIB = True
except ImportError:
    USE_TALIB = False
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

# https://github.com/polygon-io/client-python/blob/master/examples/websocket/stocks-ws.py

In [39]:
def fetch_polygon_data(ticker: str, api_key: str,
                              start_date: date,
                              end_date: date,
                              start_time: dt_time,
                              end_time: dt_time = dt_time(16, 0),
                              multiplier: int = 1,
                              timespan: str = "minute",
                              limit: int = 50000) -> pd.DataFrame:
    """
    Fetches intraday minute-level OHLCV data for a given ticker from Polygon.io.

    Args:
        ticker (str): Stock ticker symbol (e.g., "TSLA").
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch data.
        end_date (date): The date for which to end fetch data.
        start_time (datetime.time): Start of time window in EST.
        end_time (datetime.time): End of time window in EST.
        multiplier (int): Interval multiplier (default 1 minute).
        timespan (str): Time unit ("minute", "hour", etc.).
        limit (int): Maximum number of results to retrieve.

    Returns:
        pd.DataFrame: A filtered DataFrame with renamed columns and timestamps in EST.
    """
    ET_ZONE = ZoneInfo("America/New_York")

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,
        "adjusted": "true",
        "sort": "asc",
        "limit": limit
    }

    session = requests.Session()
    response = session.get(url, params=params)
    response.raise_for_status()

    bars = response.json().get("results", [])
    df = pd.DataFrame(bars)

    if df.empty:
        return pd.DataFrame()  # Return empty DataFrame if no data

    df["TIME_UTC"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df["TIME_EST"] = df["TIME_UTC"].dt.tz_convert(ET_ZONE)
    del df["TIME_UTC"]

    # Define time window
    start_dt = datetime.combine(start_date, start_time, tzinfo=ET_ZONE)
    end_dt = datetime.combine(start_date, end_time, tzinfo=ET_ZONE)

    # Filter and rename
    df = df[(df["TIME_EST"] >= start_dt) & (df["TIME_EST"] <= end_dt)].copy()
    df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close",
        "v": "volume",
        "n": "trades",
        "vw": "vwap"
    }, inplace=True)

    return df


In [40]:

def add_candlestick_patterns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds all TA-Lib candlestick pattern columns and a summary column with detected patterns.
    
    Returns:
        DataFrame with candlestick pattern columns and a 'Detected_Patterns' summary column.
    """
    # Build dictionary of candlestick pattern functions
    cdl_patterns = {
        func: getattr(talib, func)
        for func in dir(talib)
        if func.startswith("CDL")
    }

    # Apply each pattern function to the OHLC data
    for name, pattern_func in cdl_patterns.items():
        try:
            df[name] = pattern_func(df['open'], df['high'], df['low'], df['close'])
        except Exception as e:
            print(f"⚠️ Could not compute {name}: {e}")
            df[name] = 0  # Fallback to 0 if something fails

    # Detect and summarize patterns per row
    def detect_patterns(row):
        patterns = []
        for name in cdl_patterns:
            val = row[name]
            if val != 0:
                direction = "Bullish" if val > 0 else "Bearish"
                strength = abs(val)
                patterns.append(f"{name} ({direction}, Strength={strength})")
        return ', '.join(patterns) if patterns else None

    # Add summary column
    df["Detected_Patterns"] = df.apply(detect_patterns, axis=1)

    return df

In [41]:

# def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
#     Assumes columns: 'open', 'high', 'low', 'close', 'volume'

#     Returns:
#         DataFrame with new columns added.
#     """

#     # ----------------- add candlestick patterns -----------------
#     df = add_candlestick_patterns(df)
#     # ------------------------------------------------------------

#     # Bollinger Bands
#     up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
#     df['BB_up'], df['BB_dn'] = up, dn

#     # VWAP (Cumulative approximation)
#     df["VWAP_Cum"] = (df["close"] * df["volume"]).cumsum() / df["volume"].cumsum()

#     # EMA, MACD, ATR, SMA
#     df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)

#     macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
#     df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

#     df['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
#     df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
#     df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
#     df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

#     # Candlestick pattern: Marubozu
#     df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df.open.values, df.high.values, df.low.values, df.close.values)
#     df['Bullish_marubozu'] = df['CDLMARUBOZU'] == 100
#     df['Bearish_marubozu'] = df['CDLMARUBOZU'] == -100

#     # RSI
#     df['RSI'] = talib.RSI(df.close.values, timeperiod=14)

#     # ----------------- Buy Conditions -----------------

#     # B1: Price ≥ SMA5
#     df['B_1'] = df['close'] >= df['SMA5']

#     # B2: MACD histogram increasing & MACD strong
#     df['MACD_diff'] = df['MACD'] - df['Signal']
#     df['MACD_diff_avg'] = df['MACD_diff'].rolling(15).mean()
#     df['MACD_strong'] = df['MACD_diff'] > 2 * df['MACD_diff_avg']
#     df['B_2'] = (df['MACD_hist'].diff() > 0) & df['MACD_strong']

#     # B3: RSI > smoothed RSI
#     df['RSI_smooth'] = df['RSI'].ewm(span=3, adjust=False).mean()
#     df['B_3'] = df['RSI'] > df['RSI_smooth']

#     # B4: Volume spike
#     df['Vol_avg20'] = df['volume'].rolling(20).mean()
#     df['B_4'] = df['volume'] > 1.5 * df['Vol_avg20']

#     # B5: Avoid bad momentum patterns
#     df['Cond_MACD_hist_avoid'] = (df['MACD_hist'] > 0) & (df['MACD_hist'].diff() < 0)
#     df['Cond_RSI_falling'] = df['RSI'].diff() < 0
#     df['Cond_price_lower'] = df['close'] < df['close'].shift(1)
#     df['B_5'] = df[['Cond_MACD_hist_avoid', 'Cond_RSI_falling', 'Cond_price_lower']].any(axis=1)

#     # B6: Bullish marubozu breakout above recent high with follow-through
#     N = 10
#     df['RecentHigh'] = df['high'].shift(1).rolling(N).max()
#     df['Breakout_follow_through'] = (
#         (df['high'] > df['RecentHigh']) &
#         (df['close'] > df['RecentHigh']) &
#         (df['close'] > df['open'])
#     )
#     df['B_6'] = df['Bullish_marubozu'] & df['Breakout_follow_through']

#     # Final Entry Signal
#     df['Entry_Signal'] = df['B_1'] & df['B_2'] & df['B_3'] & ~df['B_5'] & df['B_6']

#     return df

In [42]:
def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
    Returns:
        A defragmented DataFrame with new columns added.
    """
    # ─────────────── Add candlestick patterns ───────────────
    df = add_candlestick_patterns(df)

    # ─────────────── Technical Indicators ───────────────
    indicators = {}

    # Bollinger Bands
    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
    indicators['BB_up'], indicators['BB_dn'] = up, dn

    # VWAP
    indicators["VWAP_Cum"] = (df["close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # EMAs
    indicators['EMA9'] = talib.EMA(df.close.values, timeperiod=9)
    indicators['EMA20'] = talib.EMA(df.close.values, timeperiod=20)
    indicators['EMA50'] = talib.EMA(df.close.values, timeperiod=50)
    indicators['EMA200'] = talib.EMA(df.close.values, timeperiod=200)

    # MACD
    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    indicators['MACD'], indicators['Signal'], indicators['MACD_hist'] = macd, sig, hist

    # ATR and SMAs
    indicators['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
    indicators['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    indicators['SMA20'] = talib.SMA(df.close.values, timeperiod=20)
    indicators['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    indicators['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

    # RSI
    indicators['RSI'] = talib.RSI(df.close.values, timeperiod=14)

    # Marubozu
    cdl = talib.CDLMARUBOZU(df.open.values, df.high.values, df.low.values, df.close.values)
    indicators['CDLMARUBOZU'] = cdl
    indicators['Bullish_marubozu'] = cdl == 100
    indicators['Bearish_marubozu'] = cdl == -100

    # ─────────────── Buy Conditions ───────────────
    logic = {}
    logic['B_1'] = df['close'] >= indicators['SMA5']

    MACD_diff = indicators['MACD'] - indicators['Signal']
    MACD_diff_avg = pd.Series(MACD_diff).rolling(15).mean()
    MACD_strong = MACD_diff > 2 * MACD_diff_avg
    logic['B_2'] = (pd.Series(indicators['MACD_hist']).diff() > 0) & MACD_strong

    RSI_smooth = pd.Series(indicators['RSI']).ewm(span=3, adjust=False).mean()
    logic['B_3'] = indicators['RSI'] > RSI_smooth

    Vol_avg20 = df['volume'].rolling(20).mean()
    logic['B_4'] = df['volume'] > 1.5 * Vol_avg20

    cond1 = (pd.Series(indicators['MACD_hist']) > 0) & (pd.Series(indicators['MACD_hist']).diff() < 0)
    cond2 = pd.Series(indicators['RSI']).diff() < 0
    cond3 = df['close'] < df['close'].shift(1)
    logic['B_5'] = cond1 | cond2 | cond3

    N = 10
    RecentHigh = df['high'].shift(1).rolling(N).max()
    breakout = (df['high'] > RecentHigh) & (df['close'] > RecentHigh) & (df['close'] > df['open'])
    logic['B_6'] = indicators['Bullish_marubozu'] & breakout

    entry_signal = logic['B_1'] & logic['B_2'] & logic['B_3'] & ~logic['B_5'] & logic['B_6']

    # ─────────────── Crossover Buy/Sell Signal ───────────────
    prev50 = pd.Series(indicators['EMA50']).shift(1)
    prev200 = pd.Series(indicators['EMA200']).shift(1)

    cross_up = (indicators['EMA50'] > indicators['EMA200']) & (prev50 <= prev200)
    cross_down = (indicators['EMA50'] < indicators['EMA200']) & (prev50 >= prev200)
    segment = (cross_up | cross_down).cumsum()

    signal_col = pd.Series(index=df.index, dtype=object)
    tmp = pd.DataFrame({'cross_up': cross_up, 'cross_down': cross_down, 'segment': segment}, index=df.index)

    firsts = tmp[tmp['cross_up']].groupby('segment').head(1)
    signal_col[firsts.index] = 'Buy'
    lasts = tmp[tmp['cross_down']].groupby('segment').tail(1)
    signal_col[lasts.index] = 'Sell'

    # ─────────────── Combine All Columns ───────────────
    all_new = {
        **indicators,
        **logic,
        'RSI_smooth': RSI_smooth,
        'MACD_diff': MACD_diff,
        'MACD_diff_avg': MACD_diff_avg,
        'Vol_avg20': Vol_avg20,
        'RecentHigh': RecentHigh,
        'Breakout_follow_through': breakout,
        'Entry_Signal': entry_signal,
        'cross_up': cross_up,
        'cross_down': cross_down,
        'segment': segment,
        'Signal': signal_col
    }

    df_new = pd.concat([df, pd.DataFrame(all_new, index=df.index)], axis=1)

    # Optional: ensure fully defragmented copy
    df_new = df_new.copy()

    return df_new


In [43]:
from datetime import date
import time
from typing import List, Dict
import pandas as pd

def fetch_multiple_tickers_data(ticker: List[str],
                                api_key: str,
                                start_date: date,
                                end_date: date,
                                start_time: dt_time,
                                end_time: dt_time,
                                multiplier: int,
                                timespan: str,
                                limit: int ,
                                delay: float = 0.25) -> Dict[str, pd.DataFrame]:
    """
    Fetch minute-level Polygon data for multiple tickers on a given date.

    Args:
        ticker (List[str]): List of stock ticker symbols.
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch intraday data.
        delay (float): Seconds to wait between API calls (rate limit buffer).

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of ticker symbol → DataFrame.
    """
    all_data = {}

    for ticker in ticker:
        try:
            df = fetch_polygon_data(
                ticker=ticker,
                api_key=API_KEY,
                start_date=start_date,
                end_date=end_date,
                start_time=start_time,
                end_time=end_time,
                multiplier=multiplier,
                timespan=timespan,
                limit=limit
                )


            if not df.empty:
                
                df = add_indicators_and_signals(df)

                all_data[ticker] = df

            print(f"✅ {ticker} - fetched {len(df)} rows")

        except Exception as e:
            print(f"❌ Error fetching {ticker}: {e}")

        time.sleep(delay) # ⏱️ Avoid hitting rate limits

    return all_data


In [44]:
# Add example 100 tickers

# TICKERS = ["AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA", "META", "NFLX", "INTC", "AMD", 
#            "BA", "CRM", "DIS", "ADBE", "PYPL", "CSCO", "PEP", "KO", "T", "VZ",
#            "WMT", "COST", "MCD", "NKE", "QCOM", "ORCL", "IBM", "GE", "CAT", "HON",
#            "JNJ", "PFE", "MRK", "LLY", "ABBV", "BMY", "CVX", "XOM", "SLB", "COP",
#            "JPM", "BAC", "WFC", "GS", "MS", "BLK", "TGT", "LOW", "HD", "SBUX",
#            "UBER", "LYFT", "ABNB", "SHOP", "SQ", "ROKU", "ZM", "PLTR", "SNOW", "NET",
#            "PANW", "CRWD", "ZS", "OKTA", "DDOG", "DOCU", "F", "GM", "RIVN", "LCID",
#            "TSM", "ASML", "AVGO", "TXN", "MU", "INTU", "ADSK", "EA", "ATVI", "TTWO",
#            "NOK", "ERIC", "V", "MA", "AXP", "PYPL", "SOFI", "HOOD", "ROBIN", "COIN",
#            "SPOT", "BIDU", "BABA", "JD", "PDD", "TME", "IQ", "YUMC", "NTES", "LI"]

TICKERS = ["SRFM","PLTR","AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA"]

In [45]:
# Date range 
start_date = date(2025, 7, 11)
end_date = date(2025, 7, 11)

# Time window for filtering intraday bars (Eastern Time)
start_time = dt_time(4, 0)   # 4:00 AM ET
end_time = dt_time(16, 0)    # 4:00 PM ET

# Granularity of data
multiplier = 30              #   e.g., 1-minute bars,  5-minute bars, etc.
timespan = "second"          #  "second", "minute", "hour", "day", etc.

# Max number of bars to retrieve per request (Polygon max = 50,000)
limit = 50000

# ⬇️ Fetch intraday data for multiple tickers from Polygon
all_data_second = fetch_multiple_tickers_data(
    ticker=TICKERS,
    api_key=API_KEY,
    start_date=start_date,
    end_date=end_date,
    start_time=start_time,
    end_time=end_time,
    multiplier=multiplier,
    timespan=timespan,
    limit=limit
)

✅ SRFM - fetched 952 rows
✅ PLTR - fetched 1222 rows
✅ AAPL - fetched 1086 rows
✅ MSFT - fetched 860 rows
✅ GOOG - fetched 910 rows
✅ TSLA - fetched 1367 rows
✅ AMZN - fetched 1146 rows
✅ NVDA - fetched 1387 rows


In [46]:

# Date range 
start_date = date(2025, 7, 11)
end_date = date(2025, 7, 11)

# Time window for filtering intraday bars (Eastern Time)
start_time = dt_time(4, 0)   # 4:00 AM ET
end_time = dt_time(16, 0)    # 4:00 PM ET
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Granularity of data
multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
timespan = "minute"          #  "second", "minute", "hour", "day", etc.

# Max number of bars to retrieve per request (Polygon max = 50,000)
limit = 50000

# ⬇️ Fetch intraday data for multiple tickers from Polygon
all_data_minute = fetch_multiple_tickers_data(
    ticker=TICKERS,
    api_key=API_KEY,
    start_date=start_date,
    end_date=end_date,
    start_time=start_time,
    end_time=end_time,
    multiplier=multiplier,
    timespan=timespan,
    limit=limit
)

✅ SRFM - fetched 523 rows
✅ PLTR - fetched 673 rows
✅ AAPL - fetched 607 rows
✅ MSFT - fetched 459 rows
✅ GOOG - fetched 499 rows
✅ TSLA - fetched 715 rows
✅ AMZN - fetched 630 rows
✅ NVDA - fetched 715 rows


In [58]:
import pandas as pd
from IPython.display import display, HTML

# Get Tesla data
df = all_data_minute["TSLA"].copy()

# Filter Buy and Sell signals
df_buy = df[df["Signal"] == "Buy"][["TIME_EST", "close", "Signal"]]
df_sell = df[df["Signal"] == "Sell"][["TIME_EST", "close", "Signal"]]

# Add Type column for styling
df_buy["Type"] = "Buy"
df_sell["Type"] = "Sell"

# Combine both
final_table = pd.concat([df_buy, df_sell], ignore_index=True)

# Save a copy without "Type" for display
final_table_display = final_table.drop(columns=["Type"])

# Insert a fake top row for title
title_row = pd.DataFrame([["This is Minute Table", "", ""]], columns=final_table_display.columns)
final_with_title = pd.concat([title_row, final_table_display], ignore_index=True)

# Apply styling
def style_rows(row):
    if row.name == 0:  # Title row
        return ['font-weight: bold; text-align: center; background-color: lightblue'] * len(row)
    else:
        idx = row.name - 1  # Adjust for title row
        signal_type = final_table.iloc[idx]["Type"]
        if signal_type == "Buy":
            return ['background-color: lightgreen'] * len(row)
        elif signal_type == "Sell":
            return ['background-color: lightcoral'] * len(row)
        else:
            return [''] * len(row)

styled = final_with_title.style.apply(style_rows, axis=1)

# Display styled table with title row
display(styled)


,TIME_EST,close,Signal
0,This is Minute Table,,
1,2025-07-11 08:03:00-04:00,309.100000,Buy
2,2025-07-11 10:00:00-04:00,309.595300,Buy
3,2025-07-11 12:13:00-04:00,309.190000,Buy
4,2025-07-11 08:56:00-04:00,308.480000,Sell
5,2025-07-11 11:08:00-04:00,307.960800,Sell


In [59]:
import pandas as pd
from IPython.display import display

# Get Tesla data
df = all_data_minute["TSLA"].copy()

# Filter only rows where Signal is Buy or Sell
df_signals = df[df["Signal"].isin(["Buy", "Sell"])][["TIME_EST", "close", "Signal"]].copy()

# Insert a title row
title_row = pd.DataFrame([["This is Minute Table", "", ""]], columns=df_signals.columns)
df_with_title = pd.concat([title_row, df_signals], ignore_index=True)

# Styling function
def style_signals(row):
    if row.name == 0:
        return ['font-weight: bold; text-align: center; background-color: lightblue'] * len(row)
    elif row["Signal"] == "Buy":
        return ['background-color: lightgreen'] * len(row)
    elif row["Signal"] == "Sell":
        return ['background-color: lightcoral'] * len(row)
    else:
        return [''] * len(row)

# Display styled table
styled = df_with_title.style.apply(style_signals, axis=1)
display(styled)


,TIME_EST,close,Signal
0,This is Minute Table,,
1,2025-07-11 08:03:00-04:00,309.100000,Buy
2,2025-07-11 08:56:00-04:00,308.480000,Sell
3,2025-07-11 10:00:00-04:00,309.595300,Buy
4,2025-07-11 11:08:00-04:00,307.960800,Sell
5,2025-07-11 12:13:00-04:00,309.190000,Buy


In [60]:
df_signals = df[df["Signal"].isin(["Buy", "Sell"])][["TIME_EST", "close", "Signal"]].copy()
display(df_signals)

,TIME_EST,close,Signal
237,2025-07-11 08:03:00-04:00,309.1000,Buy
290,2025-07-11 08:56:00-04:00,308.4800,Sell
354,2025-07-11 10:00:00-04:00,309.5953,Buy
422,2025-07-11 11:08:00-04:00,307.9608,Sell
487,2025-07-11 12:13:00-04:00,309.1900,Buy


In [ ]:
from datetime import timedelta
from IPython.display import display, HTML

last_minutes_window = 15

def highlight_entry_rows(row):
    """Highlight rows in Table 2 where Entry_Signal is True."""
    if row.get("Entry_Signal", False):
        return ['background-color: #d1f7c4'] * len(row)
    return [''] * len(row)

for ticker in all_data_minute:
    df_min = all_data_minute.get(ticker, pd.DataFrame())
    df_sec = all_data_second.get(ticker, pd.DataFrame())

    if df_min.empty or "TIME_EST" not in df_min.columns:
        print(f"⚠️ Skipping {ticker} — no TIME_EST or data in minute frame")
        continue

    # ─────────────────────────────────────
    # Table 1 — Entry signals from minute data
    df_signal_min = df_min[df_min["Signal"].isin(["Buy", "Sell"])][["TIME_EST", "close", "Signal"]].copy()
    html_table_signal_min = df_signal_min.to_html(index=False)

    # ─────────────────────────────────────
    # Table 2 — Last X min detected patterns from minute data
    if "Detected_Patterns" in df_min.columns:
        end_time = df_min["TIME_EST"].max()
        start_time = end_time - timedelta(minutes=last_minutes_window)

        recent = df_min[(df_min["TIME_EST"] >= start_time) & (df_min["TIME_EST"] <= end_time)].copy()
        recent = recent.dropna(subset=["Detected_Patterns"])
        recent = recent[["TIME_EST", "close", "Detected_Patterns", "Entry_Signal"]]
    else:
        recent = pd.DataFrame(columns=["TIME_EST", "close", "Detected_Patterns", "Entry_Signal"])

    styled_recent = recent.style.apply(highlight_entry_rows, axis=1).hide(axis='index')

    # ─────────────────────────────────────
    # Table 3 — Entry signals from second data
    if not df_sec.empty and "Signal" in df_sec.columns:
        df_signal_sec = df_sec[df_sec["Signal"].isin(["Buy", "Sell"])][["TIME_EST", "close", "Signal"]].copy()
        html_table_signal_sec = df_signal_sec.to_html(index=False)
    else:
        html_table_signal_sec = "<p style='color:gray;'>No second-resolution signal data</p>"

    # ─────────────────────────────────────
    # Display layout: Table 1 (min), Table 3 (sec), Table 2 (patterns)
    html_output = f"""
    <h3>📈 {ticker}</h3>
    <div style="display: flex; gap: 40px;">
        <div>
            <h4>🚀 Entry Signals — Minute (Table 1)</h4>
            {html_table_signal_min}
            <h4 style="margin-top: 40px;">🔁 Entry Signals — Second (Table 3)</h4>
            {html_table_signal_sec}
        </div>
        <div>
            <h4>🕒 Last {last_minutes_window} min Patterns (Table 2)</h4>
            {styled_recent.to_html()}
        </div>
    </div>
    <hr style="margin-top:30px; margin-bottom:30px;">
    """

    display(HTML(html_output))


TIME_EST,close,Signal
2025-07-11 14:50:00-04:00,142.62,Sell
TIME_EST,close,Signal
2025-07-11 07:16:00-04:00,140.8700,Buy
2025-07-11 08:28:00-04:00,140.6100,Sell
2025-07-11 08:50:00-04:00,140.9600,Buy
2025-07-11 11:13:30-04:00,142.3188,Sell
2025-07-11 12:24:30-04:00,143.4300,Buy
2025-07-11 13:39:00-04:00,142.9301,Sell
2025-07-11 14:16:30-04:00,143.6000,Buy
2025-07-11 14:37:30-04:00,143.3000,Sell


TIME_EST,close,Signal
2025-07-11 09:38:00-04:00,210.0600,Sell
2025-07-11 10:07:00-04:00,211.4750,Buy
2025-07-11 11:17:00-04:00,210.5588,Sell
2025-07-11 13:10:00-04:00,211.1450,Buy
TIME_EST,close,Signal
2025-07-11 09:33:30-04:00,210.540,Sell
2025-07-11 10:04:30-04:00,211.030,Buy
2025-07-11 10:58:30-04:00,211.000,Sell
2025-07-11 13:00:00-04:00,210.955,Buy
2025-07-11 15:02:00-04:00,211.085,Sell


TIME_EST,close,Signal
2025-07-11 08:03:00-04:00,309.1000,Buy
2025-07-11 08:56:00-04:00,308.4800,Sell
2025-07-11 10:00:00-04:00,309.5953,Buy
2025-07-11 11:08:00-04:00,307.9608,Sell
2025-07-11 12:13:00-04:00,309.1900,Buy
TIME_EST,close,Signal
2025-07-11 06:05:30-04:00,308.3900,Sell
2025-07-11 07:31:00-04:00,308.9000,Buy
2025-07-11 08:37:30-04:00,308.5400,Sell
2025-07-11 08:41:00-04:00,308.9400,Buy


TIME_EST,close,Signal
2025-07-11 14:09:00-04:00,165.75,Sell
TIME_EST,close,Signal
2025-07-11 05:59:00-04:00,162.6200,Sell
2025-07-11 06:52:30-04:00,162.9100,Buy
2025-07-11 06:58:00-04:00,162.7600,Sell
2025-07-11 06:59:00-04:00,162.9300,Buy
2025-07-11 08:41:30-04:00,163.0700,Sell
2025-07-11 08:58:30-04:00,163.2099,Buy
2025-07-11 09:22:00-04:00,163.0201,Sell
2025-07-11 09:25:30-04:00,163.2600,Buy


In [62]:
from datetime import timedelta
from IPython.display import display, HTML

last_minutes_window = 15

def highlight_entry_rows(row):
    """Highlight entire row if Entry_Signal is True."""
    if row.get("Entry_Signal", False):
        return ['background-color: #d1f7c4'] * len(row)  # light green
    return [''] * len(row)

for ticker, df in all_data_minute.items():
    if df.empty or "TIME_EST" not in df.columns:
        print(f"⚠️ Skipping {ticker} — no TIME_EST or data")
        continue

    # ─────────────────────────────────────────────
    # Table 1 — Entry signals
    # df_signal = df[df["Entry_Signal"]].copy()
    # df_signal = df_signal[["TIME_EST", "close"]]
    df_signal = df[df["Signal"].isin(["Buy", "Sell"])][["TIME_EST", "close", "Signal"]].copy()

    # ─────────────────────────────────────────────
    # Table 2 — Last X minutes of Detected Patterns
    if "Detected_Patterns" in df.columns:
        end_time = df["TIME_EST"].max()
        start_time = end_time - timedelta(minutes=last_minutes_window)

        recent = df[(df["TIME_EST"] >= start_time) & (df["TIME_EST"] <= end_time)].copy()
        recent = recent.dropna(subset=["Detected_Patterns"])
        recent = recent[["TIME_EST", "close", "Detected_Patterns", "Entry_Signal"]]
    else:
        recent = pd.DataFrame(columns=["TIME_EST", "close", "Detected_Patterns", "Entry_Signal"])
    # ─────────────────────────────────────────────
    # Format HTML with row highlighting in Table 2
    styled_recent = recent.style.apply(highlight_entry_rows, axis=1).hide(axis='index')

    html_output = f"""
    <h3>📈 {ticker}</h3>
    <div style="display: flex; gap: 40px;">
        <div>
            <h4>🚀 Entry Signals</h4>
            {df_signal.to_html(index=False)}
        </div>
        <div>
            <h4>🕒 Last {last_minutes_window} min Patterns</h4>
            {styled_recent.to_html()}
        </div>
    </div>
    <hr style="margin-top:30px; margin-bottom:30px;">
    """

    display(HTML(html_output))


In [ ]:
# from datetime import timedelta
# from IPython.display import display, HTML

# for ticker, df in all_data.items():
#     if "Detected_Patterns" not in df.columns:
#         print(f"⚠️ Skipping {ticker} - no Detected_Patterns column")
#         continue

#     if df.empty or "TIME_EST" not in df.columns:
#         print(f"⚠️ Skipping {ticker} - missing TIME_EST or data")
#         continue

#     # Get the last timestamp in the dataset
#     end_time = df["TIME_EST"].max()
#     start_time = end_time - timedelta(minutes=10)

#     # Filter to last 10 minutes
#     recent = df[(df["TIME_EST"] >= start_time) & (df["TIME_EST"] <= end_time)].copy()

#     # Keep only rows where patterns are detected
#     recent = recent.dropna(subset=["Detected_Patterns"])

#     if recent.empty:
#         print(f"🔍 {ticker} — No patterns detected in last 10 minutes.")
#         continue

#     # Display as clean HTML
#     display(HTML(f"<h3>📊 {ticker} — Detected Patterns (Last 10 min)</h3>"))
#     display(HTML(recent[["TIME_EST", "close", "Detected_Patterns"]].to_html(index=False)))


In [ ]:
# from IPython.display import display, HTML

# for ticker, df in all_data.items():
#     # Filter only rows with entry signal
#     df_signal = df.loc[df["Entry_Signal"], ["TIME_EST","close"]]

#     # Only show table if at least one signal is present
#     if not df_signal.empty:
#         display(HTML(f"<h4>{ticker}</h4>"))
#         display(df_signal)
